# Fine-tune a YOLOv8 detection model

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks. As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available.
This notebook expects that the dataset created in the notebook [01_Dataset](./01_Dataset.ipynb) has been saved to disk.

### Setup

To start, check GPU support.

In [1]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.1+cu121 (NVIDIA RTX A6000)


### Load dataset

We'll first load the dataset created on disk.

In [2]:
from pathlib import Path

dataset_dir = Path() / "dataset"

In [3]:
import fiftyone as fo

# The splits to load
splits = ["train", "val", "test"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset()
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |███████████████| 2668/2668 [2.4s elapsed, 0s remaining, 1.1K samples/s]      
 100% |█████████████████| 334/334 [302.6ms elapsed, 0s remaining, 1.1K samples/s]     
 100% |█████████████████| 333/333 [411.4ms elapsed, 0s remaining, 809.5 samples/s]     


### Train a YoloV8 model

We can now train a YoloV8 base model on our exported dataset. We use the `yolov8m.pt` (nano) model, which provides a good comprise between size and speed, but other models are available from ultralytics. Simply replace `yolov8m.pt` with the [right model](https://docs.ultralytics.com/models/yolov8/#key-features).

In [4]:
from adomvi.yolo.yolo import train

results_train = train("yolov8m.pt", data=dataset_dir / "dataset.yaml", epochs=60, batch=64, device=1)

#Load the path of the training model results
results_train_dir = Path(results_train.save_dir)

New https://pypi.org/project/ultralytics/8.2.45 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.38 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:1 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=dataset/dataset.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=1, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning /home/jrenault/workspace/adomvi2/notebooks/dataset/labels/train... 2668 images, 44 backgrounds, 0 corrupt: 100%|████

train: New cache created: /home/jrenault/workspace/adomvi2/notebooks/dataset/labels/train.cache



val: Scanning /home/jrenault/workspace/adomvi2/notebooks/dataset/labels/val... 334 images, 4 backgrounds, 0 corrupt: 100%|██████████

val: New cache created: /home/jrenault/workspace/adomvi2/notebooks/dataset/labels/val.cache


...

      60/60      25.6G     0.3827     0.2829     0.9801         53        640: 100%|██████████| 42/42 [00:41<00:00,  1.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.25s/

                   all        334        434      0.829      0.754      0.836      0.718



60 epochs completed in 0.804 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.38 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:1 (NVIDIA RTX A6000, 48677MiB)
Model summary (fused): 218 layers, 25842076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.43s/


                   all        334        434      0.848      0.743      0.868      0.732
                   AFV        257        349      0.985      0.571      0.904      0.677
                   APC         52         61      0.949      0.656       0.86      0.719
                   MEV          5          5      0.603        0.8       0.76      0.656
                   LAV         17         19      0.856      0.947      0.947      0.876
Speed: 0.2ms preprocess, 4.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train3


### Use the fine-tuned model to predict detections

Once our model is trained, we can use it to predict detections on our test dataset. Simply pass the path to the best model weights and the directory containing the test images to the `predict` function.

In [5]:
from adomvi.yolo.yolo import predict

best_model = Path() / results_train_dir / "weights/best.pt"
results_predict = predict(best_model, source=dataset_dir / "images/test")

# Load the path of the prediction model results
results_predict_dir = Path(results_predict[0].save_dir)


image 1/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/00f772b75be12ddc.jpg: 480x640 1 AFV, 206.0ms
image 2/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/01aa441c93747718.jpg: 448x640 1 AFV, 172.4ms
image 3/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/01c1bff29e25b149.jpg: 320x640 2 AFVs, 175.2ms
image 4/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/01d517af269faac1.jpg: 416x640 1 AFV, 177.5ms
image 5/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/025457c66e740c30.jpg: 448x640 2 AFVs, 10.2ms
image 6/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/028aaf856e96f775.jpg: 640x640 1 AFV, 1 MEV, 11.6ms
image 7/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/031f4ced45f48697.jpg: 512x640 1 AFV, 178.2ms
image 8/333 /home/jrenault/workspace/adomvi2/notebooks/dataset/images/test/044a3e7c7b006202.jpg: 640x640 1 AFV, 19.9ms
image 9/333 /home/jrenault/worksp

### Evaluate model predictions

We can first evaluate our model by loading the predictions in a Fiftyone dataset view of our test data. We select the images tagged as part of the test set in our dataset.

In [6]:
# The test split of the dataset
test_view = dataset.match_tags("test")

Then we can load the model predictions as fields of the images in our test view.

In [7]:
from adomvi.yolo.utils import add_yolo_detections

prediction_field = "yolov8"
predictions_dir = Path() / results_predict_dir / "labels"
add_yolo_detections(test_view, prediction_field=prediction_field, predictions_dir=predictions_dir, class_list=["AFV", "APC", "MEV", "LAV"])

Finally, we can evaluate our model's predictions and print the mean Average Precision (mAP).

In [8]:
detection_results = test_view.evaluate_detections(
    prediction_field, 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

Evaluating detections...
 100% |█████████████████| 333/333 [1.3s elapsed, 0s remaining, 253.6 samples/s]         
Performing IoU sweep...
 100% |█████████████████| 333/333 [1.7s elapsed, 0s remaining, 208.9 samples/s]         


In [9]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

mAP = 0.5429961992710977


In [10]:
detection_results.print_report()

              precision    recall  f1-score   support

         AFV       0.82      0.79      0.80       377
         APC       0.73      0.69      0.71        64
         LAV       0.47      0.69      0.56        26
         MEV       0.15      0.75      0.25         4

   micro avg       0.75      0.77      0.76       471
   macro avg       0.54      0.73      0.58       471
weighted avg       0.78      0.77      0.77       471

